In [15]:
import xml.etree.ElementTree as ET
import pandas as pd
from tqdm import tqdm
import os

In [17]:
%%script false --no-raise-error

activity_files = os.listdir('./activities')

df = pd.DataFrame()

for act in activity_files:
    tree = ET.parse('./activities/' + act)
    root = tree.getroot()

    ns = {'default': 'http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2',
          'ext': 'http://www.garmin.com/xmlschemas/ActivityExtension/v2'}

    trackpoints = root.findall('.//default:Trackpoint', ns)

    prev_altitude = None
    prev_distance = None

    for tp in tqdm(trackpoints):
        time = tp.find('default:Time', ns).text
        latitude = tp.find('default:Position/default:LatitudeDegrees', ns).text
        longitude = tp.find('default:Position/default:LongitudeDegrees', ns).text
        altitude = float(tp.find('default:AltitudeMeters', ns).text)
        distance = float(tp.find('default:DistanceMeters', ns).text)
        heart_rate = tp.find('default:HeartRateBpm/default:Value', ns).text
        speed_kmh = float(tp.find('default:Extensions/ext:TPX/ext:Speed', ns).text) * 3.6

        df = pd.concat([df, pd.DataFrame([[time, latitude, longitude, altitude, distance, heart_rate, speed_kmh]])])

        prev_altitude = altitude
        prev_distance = distance

    df['Activity'] = act

df.columns = ['Time', 'Latitude', 'Longitude', 'Altitude', 'Distance', 'HeartRate', 'Speed', 'Activity']

# Save to CSV
df.to_csv('activities.csv', index=False)

Couldn't find program: 'false'
